# Calculate GCaMP DFF

### 1) Read in the autofluorescence, GCaMP, and shock csv files and return auto, gcamp, and shock pandas DataFrames.

In [2]:
#import packages 
import os as os #os
import pandas as pd #pandas
import numpy as np #numpy
import scipy as scipy #scipy
import matplotlib.lines as mlines #matplotlib
import matplotlib.pyplot as plt #matplotlib
plt.style.use('ggplot') #emulate ggplot from R
#% matplotlib inline 
#view plots in jupyter notebook

#### *User Input Required Below*

In [4]:
#***change working directory, ID, session, and number of trials**** 
#***These are the only details you need to change to run the whole script***

#IMPORTANT - to change working directory, use os.chdir(path)
os.chdir('C:\\')
working_directory = os.getcwd() 
print(working_directory)
ID = ''
session = ''


C:\


#### *User Input Required Below*

In [61]:
#read in the autofluorescence, GCaMP, and cue csv files and return auto, gcamp, and shock pandas DataFrames
#IMPORTANT - to change file name, format as ('file name.csv')
auto = pd.read_csv(ID + '_' + session +'_AF.csv') 
gcamp = pd.read_csv(ID + '_' + session + '_GC.csv')
shock = pd.read_csv(ID + '_' + session + '_cue.csv')

### 2) Combine the time column and the auto, gcamp, and shock d0 columns to create a master pandas DataFrame. Write out the master pandas DataFrame as a csv file to the working directory.

In [62]:
#make auto, gcamp, and shock column headings lowercase
auto.columns = auto.columns.str.lower()
gcamp.columns = gcamp.columns.str.lower()
shock.columns = shock.columns.str.lower()
#absolute value of shock d0 column values
shock.d0 = shock.d0.abs()
#combine time column and auto, gcamp, and shock d0 columns to create master pandas DataFrame
master = pd.concat([auto['time'], auto['d0'], gcamp['d0'], shock['d0']], axis = 1, keys = ['time', 'auto', 'gcamp', 'shock'])
#write out master as a csv file to working directory
master.to_csv(ID + '_' + session + '_master.csv')

### 3) Determine the data range for the calculations. Create a master_input pandas DataFrame.

In [64]:
#determine the rows in which shock occurs
shock_rows = master.loc[master.shock > .75].index[:].tolist()
#determine the rows in which shock onset occurs
shock_onset_rows = [shock_rows[0]]
for i in range(1, len(shock_rows)):
    if shock_rows[i] > shock_rows[i - 1] + 1: 
        shock_onset_rows.append(shock_rows[i])

file_num = 1 #set file number to start at 1 initially 
shock_onset_rows = shock_onset_rows[0:len(shock_rows)]
print('shock_onset_rows =' , shock_onset_rows)

shock_onset_rows = [591, 1055, 1649, 2257, 3496, 3946, 4961, 5912, 6278, 7010, 7652, 8272, 8417, 9141, 9965, 10510, 10666, 11503, 11785, 12158, 12918, 13704, 14390, 15089, 15451, 15877, 16059, 16616, 16755, 17349, 17534, 18136, 18237, 18312, 18948, 19203, 19503, 19641, 19738, 19820, 19870, 20286, 20817, 21124, 21276, 21774, 21942, 22023, 22679, 23411, 24024, 24282, 24532, 24606, 24927, 25748, 25884, 26503, 27161, 27657]


#### *User Input Required Below*

In [65]:
for num in shock_onset_rows: 
    #create shock_onset
    shock_onset = num #IMPORTANT - can change number to any of the numbers in shock_onset_rows
    if shock_onset in shock_onset_rows:
        print('shock_onset =', shock_onset)
    else:
        raise ValueError('shock_onset not found in shock_onset_rows')
    #create begin_input 
    begin_input = shock_onset - 120 #IMPORTANT - can change number to any number of rows before shock onset
    print('begin_input =', begin_input)
    #create last_row
    last_row = len(master) - shock_onset - 1
    #create end_inputD7_Ext3_091021_AF
    end_input = shock_onset + 240 #IMPORTANT - can change number to any number of rows after shock onset or to last_row for the last row in the data set
    print('end_input =', end_input)
    #create file name for future files
    file_num_str = str(file_num) #change file number to string so it can be added to the file name
    file_name = ID + '_' + session + '_Stim_T'+ file_num_str #IMPORTANT - to change file name, format as 'file name8
    #create master_input pandas DataFrame
    master_input = master[begin_input:end_input + 1]
    
    ##### 4) Determine auto/gcamp linear trendline equations. Create auto/gcamp scatter plots with auto/gcamp linear trendlines. Save the auto/gcamp plots as PDFs to the working directory.
    #create master_trendlines pandas DataFrame
    master_trendlines = master_input.loc[begin_input:shock_onset - 1]
    #reset master_trendlines row index 
    master_trendlines = master_trendlines.reset_index(drop = True)
    #create x_master_trendlines (ranging from 1 to # rows in master_trendlines) pandas DataFrame
    x_range_master_trendlines = master_trendlines.axes[0] - (master_trendlines.axes[0][0] - 1)
    x_master_trendlines = pd.DataFrame({'x': x_range_master_trendlines})
    #add x_master_trendlines to master_trendlines 
    master_trendlines = pd.concat([x_master_trendlines, master_trendlines], axis=1, join='inner') 
    
    #determine auto linear trendline equation
    from pylab import *
    (a, b) = polyfit(master_trendlines.x, master_trendlines.auto, 1)
    auto_linear_trendline_equation = 'y = ' + str(round(a, 5)) + 'x + ' + str(round(b, 5))
    
    #create auto scatter plot with auto linear trendline
    plt.scatter(master_trendlines.x, master_trendlines.auto, color = 'blue', s = 10)
    auto_trendline_values = polyval([a,b], master_trendlines.x)
    plt.plot(master_trendlines.x, auto_trendline_values, linewidth = 3, color = 'red')
    plt.title(auto_linear_trendline_equation, fontsize = 10, y = 0.9)
    plt.xlabel('x')
    plt.ylabel('autofluorescence')
    #save auto scatter plot as PDF to working directory
    plt.savefig('auto_plot_' + file_name + '.pdf')
    
    #determine gcamp linear trendline equation
    from pylab import *
    (c, d) = polyfit(master_trendlines.x, master_trendlines.gcamp, 1)
    gcamp_linear_trendline_equation = 'y = ' + str(round(c, 5)) + 'x + ' + str(round(d, 5))
    
    #create gcamp scatter plot with gcamp linear trendline
    plt.scatter(master_trendlines.x, master_trendlines.gcamp, color = 'blue', s = 10)
    gcamp_trendline_values = polyval([c,d], master_trendlines.x)
    plt.plot(master_trendlines.x, gcamp_trendline_values, linewidth = 3, color = 'red')
    plt.title(gcamp_linear_trendline_equation, fontsize = 10, y = 0.9)
    plt.xlabel('x')
    plt.ylabel('gcamp')
    #save gcamp scatter plot as PDF to working directory
    plt.savefig('gcamp_plot_' + file_name + '.pdf')
    
    #create master_calculations pandas DataFrame
    master_calculations = master_input
    #reset master_calculations row index 
    master_calculations = master_calculations.reset_index(drop = True)
    #create x_master_calculations (ranging from 1 to # rows in master_calculations) pandas DataFrame
    x_range_master_calculations = master_calculations.axes[0] - (master_calculations.axes[0][0] - 1)
    x_master_calculations = pd.DataFrame({'x': x_range_master_calculations})
    #add x_master_calculations to master_calculations 
    master_calculations = pd.concat([x_master_calculations, master_calculations], axis = 1, join = 'inner')
    
    #create auto_trendline_y pandas DataFrame 
    auto_y_list = []
    for x in x_range_master_calculations:
        y = a*x + b 
        auto_y_list.append(y)
    auto_y_array = np.array(auto_y_list)
    auto_trendline_y = pd.DataFrame({'auto_trendline_y': auto_y_array})
    #add auto_trendline_y to master_calculations
    master_calculations = pd.concat([master_calculations, auto_trendline_y], axis = 1, join = 'inner')
    #create gcamp_trendline_y pandas DataFrame
    gcamp_y_list = []
    for x in x_range_master_calculations:
        y = a*x + d 
        gcamp_y_list.append(y)
    gcamp_y_array = np.array(gcamp_y_list)
    gcamp_trendline_y = pd.DataFrame({'gcamp_trendline_y': gcamp_y_array})
    #add gcamp_trendline_y to master_calculations
    master_calculations = pd.concat([master_calculations, gcamp_trendline_y], axis = 1, join = 'inner')
    
    #subtract auto_trendline_y from auto to create auto_fit column in master_calculations
    master_calculations['auto_fit'] = master_calculations['auto'] - master_calculations['auto_trendline_y']
    #subtract gcamp_trendline_y from gcamp to create gcamp_fit column in master_calculations
    master_calculations['gcamp_fit'] = master_calculations['gcamp'] - master_calculations['gcamp_trendline_y']
    
    #add gcamp_trendline y-intercept to auto_fit to create auto_fit column in master_calculations
    master_calculations['auto_final'] = d + master_calculations['auto_fit']
    #add gcamp_trendline y-intercept to gcamp_fit to create gcamp_fit column in master_calculations
    master_calculations['gcamp_final'] = d + master_calculations['gcamp_fit']

    #calculate delta f/f (dff) and create dff column in master_calculations
    master_calculations['dff']= ((master_calculations['gcamp_final'] - master_calculations['auto_final'])/master_calculations['auto_final'])*100
    
    #write out master_calculations as a csv file to working directory
    master_calculations.to_csv('master_calculations_' + file_name + '.csv') 
    
    #determine the rows in which shock occurs
    shock_rows_dff = master_calculations.loc[master_calculations.shock < 1].index[:].tolist()
    #determine the rows in which shock onset occurs
    shock_onset_rows_dff = [shock_rows_dff[0]]
    for i in range(2, len(shock_rows_dff)):
        if shock_rows_dff[i] > shock_rows_dff[i - 1] + 1: 
            shock_onset_rows_dff.append(shock_rows_dff[i])
    #determine the times in which shock onset occurs
    shock_onset_time_dff = list(master_calculations.time.loc[shock_onset_rows_dff])
    #create dff line plot
    fig, ax = plt.subplots()
    ax.plot(master_calculations.time, master_calculations.dff, color = 'blue')
    ax.set_xlabel('time (sec)')
    ax.set_ylabel('delta f/f')
    x = 0
    while x < len(shock_onset_time_dff):
        ax.annotate(' ', xy =(shock_onset_time_dff[x], min(master_calculations.dff)), arrowprops = dict(facecolor = 'black', shrink = 0.05))
        x = x + 1
    arrow = mlines.Line2D([], [], color = 'black', marker = '^', markersize = 12, label = 'cue onset')
    ax.legend(handles = [arrow])
    #while x < len(shock_onset_time_dff):
        #ax.annotate(' ', xy =([shock_onset_time_dff[x] + 30], min(master_calculations.dff)), arrowprops = dict(facecolor = 'yellow', shrink = 0.05))
        #x = x + 1
    #arrow2 = mlines.Line2D([], [], color = 'yellow', marker = '^', markersize = 12, label = 'cue offset')
    #ax.legend(handles = [arrow, arrow2])
    #save dff line plot as PDF to working directory
    fig.savefig('dff_plot_' + file_name + '.pdf') 
    
    #Necessary for iterative code
    matplotlib.pyplot.close('all')
    file_num += 1 #increase file number by 1 for next file

shock_onset = 591
begin_input = 551
end_input = 631
shock_onset = 1055
begin_input = 1015
end_input = 1095
shock_onset = 1649
begin_input = 1609
end_input = 1689
shock_onset = 2257
begin_input = 2217
end_input = 2297
shock_onset = 3496
begin_input = 3456
end_input = 3536
shock_onset = 3946
begin_input = 3906
end_input = 3986
shock_onset = 4961
begin_input = 4921
end_input = 5001
shock_onset = 5912
begin_input = 5872
end_input = 5952
shock_onset = 6278
begin_input = 6238
end_input = 6318
shock_onset = 7010
begin_input = 6970
end_input = 7050
shock_onset = 7652
begin_input = 7612
end_input = 7692
shock_onset = 8272
begin_input = 8232
end_input = 8312
shock_onset = 8417
begin_input = 8377
end_input = 8457
shock_onset = 9141
begin_input = 9101
end_input = 9181
shock_onset = 9965
begin_input = 9925
end_input = 10005
shock_onset = 10510
begin_input = 10470
end_input = 10550
shock_onset = 10666
begin_input = 10626
end_input = 10706
shock_onset = 11503
begin_input = 11463
end_input = 11543
sho

In [66]:
#Concatenation step 
#Make sure you have number_trials set to the correct number at the beginning of the script

#number_trials = [x+1 for x in range(0,len(shock_rows))] #change range number to reflect your number of trials
#print(number_trials)

rows = [x+1 for x in range(0,len(shock_onset_rows))] #set at beginning of script
file_name = 'master_calculations_' + ID + '_' + session + '_Stim_T' + str(rows[0]) + '.csv' #Changes file name for each animal based on settings at begining

df = pd.read_csv(file_name)
df.drop('Unnamed: 0', axis=1, inplace=True)
df.head(3)

total_dff = pd.DataFrame()
total_dff[str(rows[0])] = df.dff
total_dff.head(3)

for row in rows:
    file_name = 'master_calculations_' + ID + '_' + session + '_Stim_T' + str(row) + '.csv' #Changes file name for each animal
    df = pd.read_csv(file_name)
    df.drop('Unnamed: 0', axis=1, inplace=True)
    total_dff[str(row)] = df.dff
    
total_dff.head(5)

total_dff['avg'] = total_dff.mean(axis=1)

total_dff.to_csv(ID + '_' + session + '_final.csv') #Save to new file. 

In [16]:
print(d)

291.4410305165385
